In [5]:
import numpy as np
import pandas as pd
import dask.dataframe as dk
import tensorflow as tf
file_path = "npm_features_dataset.csv"
out_path =  "npm_shuffled.csv"

df = dk.read_csv(file_path)
# df = df.drop(columns=['Variance', 'Weight'])
print(df.dtypes)
print(df.head(10))

Tổng số mẫu: 8984
Phân bố label:
label
0    2999
1    5985
Name: count, dtype: int64
Số lượng label liên tiếp trong 1 triệu mẫu đầu: 8982


In [2]:
from dask_ml.model_selection import train_test_split 
from sklearn.utils import shuffle

def shuffle_dask_dataframe(df, random_state=None):
    return (
        df.map_partitions(
            lambda part: part.assign(__shuffle_key=np.random.RandomState(random_state).random(len(part)))
        )
        .shuffle('__shuffle_key')
        .drop(columns='__shuffle_key')
    )
    
df = shuffle_dask_dataframe(df)

print(df.dtypes)
print(df.npartitions)
print(df.head(20))

has_install_scripts               int64
has_dependencies                  int64
has_dev_dependencies              int64
num_dependencies                  int64
num_dev_dependencies              int64
num_files                         int64
total_code_size                   int64
avg_entropy                     float64
max_entropy                     float64
num_urls                          int64
num_ips                           int64
has_eval                          int64
has_child_process                 int64
has_fs_access                     int64
has_network_access                int64
has_os_access                     int64
package_name            string[pyarrow]
label                             int64
dtype: object
1
    has_install_scripts  has_dependencies  has_dev_dependencies  \
0                     0                 0                     0   
1                     0                 0                     0   
2                     0                 0                     0

In [3]:
header = True
for i in range(df.npartitions):
    df.get_partition(i).compute().to_csv(out_path, mode='a', header = header, index=False)
    header = False